# Profiling Python Code:

Profiling is the process of measuring the performance of a program to identify bottlenecks. Python provides several tools for profiling, such as cProfile and timeit.

In [ ]:
# main.py
import cProfile #Using cProfile to measure the performance of code
import pstats
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('vehicles.db')
cursor = conn.cursor()

# Function to retrieve all vehicles
def get_all_vehicles():
    cursor.execute('SELECT * FROM vehicles')
    return cursor.fetchall()

# Profile the function
def profile_get_all_vehicles():
    profiler = cProfile.Profile()
    profiler.enable()

    vehicles = get_all_vehicles()

    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('cumtime')
    stats.print_stats()

profile_get_all_vehicles()


# Using Caching Mechanisms (Redis)

Caching is a technique to store frequently accessed data in a temporary storage (cache) to reduce access time. Redis is a popular in-memory data structure store that can be used as a cache.

In [ ]:
# main.py
import redis #to store frequently accessed data and reduce access time.
import json
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('vehicles.db')
cursor = conn.cursor()

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

# Function to retrieve all vehicles with caching
def get_all_vehicles():
    # Check if data is in cache
    cached_vehicles = r.get('all_vehicles')
    if cached_vehicles:
        return json.loads(cached_vehicles)

    # If not in cache, retrieve from database
    cursor.execute('SELECT * FROM vehicles')
    vehicles = cursor.fetchall()

    # Store the result in cache
    r.set('all_vehicles', json.dumps(vehicles), ex=60)  # Cache for 60 seconds
    return vehicles

# Example usage
vehicles = get_all_vehicles()
print(vehicles)


# Understanding Time and Space Complexity:

Time and space complexity are measures of the efficiency of an algorithm. Time complexity is the computational time taken by an algorithm, while space complexity is the amount of memory it uses.

In [ ]:
# main.py
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('vehicles.db')
cursor = conn.cursor()

# Function to add a vehicle
def add_vehicle(make, model, year, price, manufacturer_id, owner_id):
    cursor.execute('''
    INSERT INTO vehicles (make, model, year, price, manufacturer_id, owner_id)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (make, model, year, price, manufacturer_id, owner_id))
    conn.commit()
    return cursor.lastrowid

# Time complexity: O(1) because inserting a record into a database has constant time complexity.
# Space complexity: O(1) because the space used for the operation is constant.

# Function to retrieve all vehicles
def get_all_vehicles():
    cursor.execute('SELECT * FROM vehicles')
    return cursor.fetchall()

# Time complexity: O(n) where n is the number of vehicles in the database. This is because we need to retrieve each vehicle
# Space complexity: O(n) because the space required to store the result grows linearly with the number of vehicles

# Example usage
manufacturer_id = 1  # Assuming a manufacturer with ID 1 exists
owner_id = 1  # Assuming an owner with ID 1 exists
vehicle_id = add_vehicle('Corolla', 'Sedan', 2020, 20000, manufacturer_id, owner_id)

vehicles = get_all_vehicles()
print(vehicles)
